## Новый подход

In [5]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional, Union, Generator
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, save_npz, load_npz
import json
import pickle
from pathlib import Path
import psutil
import os
import gc
import time
from dataclasses import dataclass
import warnings
import re
from sklearn.preprocessing import normalize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
warnings.filterwarnings('ignore')


def clean_text_for_embedding(text):
    """
    Быстрая чистка текста для эмбеддингов
    """
    if not isinstance(text, str):
        return ""
    
    text = re.sub(r'http\S+|www\S+|https\S+|@\S+', '', text, flags=re.MULTILINE)
    
    text = re.sub(r'\(\d+\)', '', text)

    text = re.sub(r'Subscribe to.*casts\.', '', text, flags=re.DOTALL)
    text = re.sub(r'Listen to.*episode.*:', '', text, flags=re.DOTALL)
    text = re.sub(r'You also can follow.*@\w+', '', text)
    
    text = re.sub(r'\n\s*\n', '\n', text)
    text = re.sub(r'\n+', ' ', text)
    
    text = ' '.join(text.split())
    
    return text.strip()


@dataclass
class EmbeddingConfig:
    """Конфигурация для генерации эмбеддингов"""
    batch_size: int = 32
    use_mmap: bool = True  
    temp_dir: str = "./temp_embeddings"
    max_memory_gb: float = 4.0  
    dtype: str = "float32"

class BaseEmbeddingGenerator(ABC):
    """Базовый абстрактный класс для генераторов эмбеддингов"""
    
    def __init__(self, config: EmbeddingConfig = None):
        self.config = config or EmbeddingConfig()
        Path(self.config.temp_dir).mkdir(parents=True, exist_ok=True)
        
    @abstractmethod
    def fit(self, texts: List[str]) -> 'BaseEmbeddingGenerator':
        """Обучение модели на текстах"""
        pass
    
    @abstractmethod
    def transform(self, texts: List[str]) -> Union[np.ndarray, csr_matrix]:
        """Преобразование текстов в эмбеддинги"""
        pass
    
    @abstractmethod
    def fit_transform(self, texts: List[str]) -> Union[np.ndarray, csr_matrix]:
        """Обучение и преобразование"""
        pass
    
    def save(self, path: str) -> None:
        """Сохранение модели"""
        save_path = Path(path)
        save_path.mkdir(parents=True, exist_ok=True)
        
        # Сохраняем конфиг
        config_path = save_path / "config.json"
        with open(config_path, 'w') as f:
            json.dump(self.config.__dict__, f)
    
    @classmethod
    def load(cls, path: str) -> 'BaseEmbeddingGenerator':
        """Загрузка модели"""
        pass
    
    def _check_memory_usage(self) -> bool:
        """Проверка использования памяти"""
        process = psutil.Process(os.getpid())
        memory_gb = process.memory_info().rss / 1024 / 1024 / 1024
        
        if memory_gb > self.config.max_memory_gb:
            print(f"Предупреждение: Использовано {memory_gb:.2f} GB памяти "
                  f"(лимит: {self.config.max_memory_gb} GB)")
            return False
        return True
    
    def _batch_generator(self, data: List, batch_size: int) -> Generator:
        """Генератор батчей"""
        for i in range(0, len(data), batch_size):
            yield data[i:i + batch_size]

In [5]:
import torch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

class DenseEmbeddingGenerator(BaseEmbeddingGenerator):
    """Генератор dense эмбеддингов"""
    
    def __init__(self,
                 model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                 device: Optional[str] = None,
                 normalize: bool = True,
                 config: EmbeddingConfig = None):
        super().__init__(config)
        
        self.model_name = model_name
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.normalize = normalize
        
        self._model = None
        self.embedding_dim = 0
    
    @property
    def model(self) -> SentenceTransformer:
        """Ленивая загрузка модели"""
        if self._model is None:
            print(f"Загрузка dense модели: {self.model_name}")
            
            self._free_memory()
            
            self._model = SentenceTransformer(
                self.model_name,
                device=self.device
            )
            
            test_embedding = self._model.encode(["test"], 
                                               convert_to_numpy=True)
            self.embedding_dim = test_embedding.shape[1]
            
            print(f"✓ Модель загружена. Размерность эмбеддингов: {self.embedding_dim}")
        
        return self._model
    
    def _free_memory(self):
        """Освобождение памяти"""
        gc.collect()
        if self.device == 'cuda' and torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    def fit(self, texts: List[str]) -> 'DenseEmbeddingGenerator':
        """Для dense моделей fit не требуется (но нужен для интерфейса)"""
        print("Dense модели не требуют обучения на данных")
        return self
    
    def transform(self, texts: List[str], 
                  save_to_file: Optional[str] = None,
                  show_progress: bool = True) -> np.ndarray:
        """Преобразование текстов в dense эмбеддинги"""
        print(f"Генерация dense эмбеддингов для {len(texts)} текстов...")
        
        if len(texts) > 50000 and self.config.use_mmap:
            return self._transform_large_mmap(texts, save_to_file, show_progress)
        
        if len(texts) > 1000:
            return self._transform_batched(texts, save_to_file, show_progress)
        
        return self._transform_small(texts, save_to_file, show_progress)
    
    def _transform_small(self, texts: List[str], 
                        save_to_file: Optional[str],
                        show_progress: bool) -> np.ndarray:
        """Обработка небольших данных"""
        embeddings = self.model.encode(
            texts,
            batch_size=self.config.batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            normalize_embeddings=self.normalize
        )
        
        if save_to_file:
            np.save(save_to_file, embeddings)
            print(f"✓ Эмбеддинги сохранены в {save_to_file}")
        
        return embeddings
    
    def _transform_batched(self, texts: List[str], 
                          save_to_file: Optional[str],
                          show_progress: bool) -> np.ndarray:
        """Батчевая обработка средних данных"""
        print("Используем батчевую обработку...")
        
        all_embeddings = []
        batch_size = self.config.batch_size
        
        if show_progress:
            pbar = tqdm(total=len(texts), desc="Генерация эмбеддингов")
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]

            batch_emb = self.model.encode(
                batch_texts,
                batch_size=batch_size,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=self.normalize
            )
            
            all_embeddings.append(batch_emb)
            
            if show_progress:
                pbar.update(len(batch_texts))
            
            if i % (batch_size * 10) == 0:
                self._check_memory_usage()
                gc.collect()
        
        if show_progress:
            pbar.close()

        embeddings = np.vstack(all_embeddings)
        
        if save_to_file:
            np.save(save_to_file, embeddings)
            print(f"✓ Эмбеддинги сохранены в {save_to_file}")
        
        return embeddings
    
    def _transform_large_mmap(self, texts: List[str], 
                             save_to_file: Optional[str],
                             show_progress: bool) -> np.ndarray:
        """Обработка очень больших данных с memory-mapped файлами"""
        print("Используем memory-mapped файлы для больших данных...")
        
        if not save_to_file:
            raise ValueError("Для больших данных требуется указать save_to_file")
        
        total_samples = len(texts)
        mmap_file = np.memmap(
            save_to_file,
            dtype=self.config.dtype,
            mode='w+',
            shape=(total_samples, self.embedding_dim)
        )
        
        batch_size = self.config.batch_size
        
        if show_progress:
            pbar = tqdm(total=total_samples, desc="Генерация эмбеддингов")
        
        for i in range(0, total_samples, batch_size):
            batch_texts = texts[i:i + batch_size]
            
            batch_emb = self.model.encode(
                batch_texts,
                batch_size=batch_size,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=self.normalize
            )
            
            mmap_file[i:i + len(batch_texts)] = batch_emb
            
            if show_progress:
                pbar.update(len(batch_texts))

            del batch_emb
            if i % (batch_size * 20) == 0:
                self._check_memory_usage()
                gc.collect()
                if self.device == 'cuda':
                    torch.cuda.empty_cache()
        
        if show_progress:
            pbar.close()
        
        mmap_file.flush()
        
        print(f"✓ Эмбеддинги сохранены в {save_to_file}")

        return np.load(save_to_file, mmap_mode='r')
    
    def fit_transform(self, texts: List[str], 
                     save_to_file: Optional[str] = None,
                     show_progress: bool = True) -> np.ndarray:
        """Обучение и преобразование"""
        self.fit(texts)
        return self.transform(texts, save_to_file, show_progress)
    
    def save(self, path: str) -> None:
        """Сохранение конфигурации модели"""
        super().save(path)
        save_path = Path(path)
        
        model_info = {
            'model_name': self.model_name,
            'device': self.device,
            'normalize': self.normalize,
            'embedding_dim': self.embedding_dim
        }
        
        model_info_path = save_path / "model_info.json"
        with open(model_info_path, 'w') as f:
            json.dump(model_info, f)
    
    @classmethod
    def load(cls, path: str) -> 'DenseEmbeddingGenerator':
        """Загрузка конфигурации модели"""
        load_path = Path(path)
        
        config_path = load_path / "config.json"
        with open(config_path, 'r') as f:
            config_dict = json.load(f)
        
        config = EmbeddingConfig(**config_dict)
        
        model_info_path = load_path / "model_info.json"
        with open(model_info_path, 'r') as f:
            model_info = json.load(f)
        
        instance = cls(
            model_name=model_info['model_name'],
            device=model_info['device'],
            normalize=model_info['normalize'],
            config=config
        )
        
        instance.embedding_dim = model_info['embedding_dim']
        
        return instance

In [7]:
class EmbeddingFactory:
    """Фабрика для создания и управления эмбеддингами"""
    
    @staticmethod
    def create_dense_generator(config: Dict[str, Any] = None) -> DenseEmbeddingGenerator:
        """Создание dense генератора"""
        default_config = {
            'model_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
            'device': None,
            'normalize': True,
            'config': EmbeddingConfig(batch_size=32)
        }
        
        if config:
            default_config.update(config)
        
        return DenseEmbeddingGenerator(
            model_name=default_config['model_name'],
            device=default_config['device'],
            normalize=default_config['normalize'],
            config=default_config['config']
        )

In [6]:
model_name = "Qwen/Qwen3-Embedding-0.6B"

In [6]:
with open('/kaggle/input/crunch/techcrunch_ai_5488_articles_20260112_1535.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)

texts = [article['text'] for article in articles]
texts_clean = [clean_text_for_embedding(text) for text in texts]
print(f"Загружено {len(texts)} статей")

# 'intfloat/multilingual-e5-large-instruct'
dense_config = {
    'model_name': model_name,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    #'device': 'cpu',
    'config': EmbeddingConfig(
        batch_size=2,
        use_mmap=True,
        max_memory_gb=4.0
    )
}

dense_gen = EmbeddingFactory.create_dense_generator(dense_config)
dense_file = "/kaggle/working/dense_embeddings.npy"
        
dense_embeddings = dense_gen.fit_transform(
    texts_clean,
    save_to_file=dense_file,
    show_progress=True
)

Загружено 5488 статей
Dense модели не требуют обучения на данных
Генерация dense эмбеддингов для 5488 текстов...
Используем батчевую обработку...


Генерация эмбеддингов:   0%|          | 0/5488 [00:00<?, ?it/s]

Загрузка dense модели: Qwen/Qwen3-Embedding-0.6B
✓ Модель загружена. Размерность эмбеддингов: 1024
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: 

In [7]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from typing import List, Dict, Set
from sklearn.preprocessing import normalize
from datetime import datetime


with open('/kaggle/input/crunch/techcrunch_ai_5488_articles_20260112_1535.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)
print(f"Статей: {len(articles)}")

with open('/kaggle/input/crunch2/questions.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)
print(f"Вопросов: {len(questions)}")

article_id_to_idx = {}
article_titles = []
for idx, article in enumerate(articles):
    article_id = article.get('id', str(idx))
    article_id_to_idx[article_id] = idx
    article_titles.append(article.get('title', ''))

dense_file = "/kaggle/input/crunch2/dense_embeddings_qwen.npy"
dense_embeddings = np.load(dense_file)
print(f"Эмбеддинги: {dense_embeddings.shape}")

# Создаем эмбеддинги для названий
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer(model_name, device=device)

title_embeddings = model.encode(
    article_titles,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)
print(f"Эмбеддинги названий: {title_embeddings.shape}")

# Нормализуем эмбеддинги
article_embeddings_norm = normalize(dense_embeddings)
title_embeddings_norm = normalize(title_embeddings)

def enhanced_dense_search(query: str, k: int = 10, 
                         text_weight: float = 0.7,
                         title_weight: float = 0.3) -> List[str]:
    """
    Улучшенный поиск с учетом текста и названий статей
    
    Args:
        query: текст запроса
        k: количество результатов
        text_weight: вес текста статьи (0-1)
        title_weight: вес названия статьи (0-1)
    """
    # Получаем эмбеддинг запроса
    query_emb = model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Поиск по тексту статей
    text_similarities = query_emb @ article_embeddings_norm.T
    
    # Поиск по названиям статей
    title_similarities = query_emb @ title_embeddings_norm.T
    
    # Комбинируем результаты с весами
    combined_scores = (text_weight * text_similarities + 
                      title_weight * title_similarities)
    
    # Топ-K индексов
    top_indices = np.argsort(-combined_scores)[:k]
    
    # Преобразуем индексы в ID
    top_ids = []
    for idx in top_indices:
        # Ищем ID по индексу
        for article_id, article_idx in article_id_to_idx.items():
            if article_idx == idx:
                top_ids.append(article_id)
                break
    
    return top_ids
    
def enhanced_time_dense_search(
    query: str, 
    k: int = 10, 
    text_weight: float = 0.6,
    title_weight: float = 0.2,
    recency_weight: float = 0.2,
    time_decay_days: int = 365  # Статьи старше года получают минимальный буст
) -> List[str]:
    """
    Улучшенный поиск с учетом текста, названий и актуальности статей
    """
    # Получаем эмбеддинг запроса
    query_emb = model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Поиск по тексту статей
    text_similarities = query_emb @ article_embeddings_norm.T
    
    # Поиск по названиям статей
    title_similarities = query_emb @ title_embeddings_norm.T
    
    # Рассчитываем временной скор
    current_time = datetime.now()
    time_scores = []
    
    for article in articles:
        try:
            pub_time = datetime.fromisoformat(article.get('published_time', '2020-01-01'))
            days_diff = (current_time - pub_time).days
            # Экспоненциальный затухающий буст (больше для свежих статей)
            time_score = np.exp(-days_diff / time_decay_days)
        except:
            time_score = 0.1  # Минимальный скор для статей без даты
        
        time_scores.append(time_score)
    
    time_scores = np.array(time_scores)
    time_scores = time_scores / time_scores.max()  # Нормализуем к [0, 1]
    
    # Комбинируем результаты
    combined_scores = (
        text_weight * text_similarities + 
        title_weight * title_similarities +
        recency_weight * time_scores
    )
    
    # Топ-K индексов
    top_indices = np.argsort(-combined_scores)[:k]
    
    # Преобразуем индексы в ID
    top_ids = []
    for idx in top_indices:
        for article_id, article_idx in article_id_to_idx.items():
            if article_idx == idx:
                top_ids.append(article_id)
                break
    
    return top_ids
    
def basic_dense_search(query: str, k: int = 10) -> List[str]:
    """Только по тексту статей"""
    query_emb = model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    similarities = query_emb @ article_embeddings_norm.T
    top_indices = np.argsort(-similarities)[:k]
    
    top_ids = []
    for idx in top_indices:
        for article_id, article_idx in article_id_to_idx.items():
            if article_idx == idx:
                top_ids.append(article_id)
                break
    
    return top_ids

# 4. Функция оценки
def evaluate_search(search_func, k_values=[1, 3, 5, 10]):
    """Оценка поиска с выводом метрик и деталей"""
    print(f"\n{'='*60}")
    print("ОЦЕНКА ПОИСКА ПО DENSE ЭМБЕДДИНГАМ")
    print(f"{'='*60}")
    
    results = []
    
    # Для каждого K считаем метрики
    for k in k_values:
        print(f"\nТОП-{k} РЕЗУЛЬТАТЫ:")
        print("-" * 80)
        
        total_precision = 0
        total_recall = 0
        total_hit = 0
        total_mrr = 0
        valid_queries = 0
        
        # Обрабатываем каждый вопрос
        for i, question_data in enumerate(questions):
            query = question_data.get("question", "")
            correct_ids = set(question_data.get("id", []))
            
            if not correct_ids:
                continue
            
            # Поиск
            found_ids = search_func(query, k=k)
            
            # Считаем метрики
            correct_found = [id for id in found_ids if id in correct_ids]
            
            # Precision@K
            precision = len(correct_found) / k if k > 0 else 0
            
            # Recall@K
            recall = len(correct_found) / len(correct_ids) if correct_ids else 0
            
            # Hit Rate@K
            hit = 1 if len(correct_found) > 0 else 0
            
            # MRR@K
            mrr = 0
            for rank, found_id in enumerate(found_ids, 1):
                if found_id in correct_ids:
                    mrr = 1.0 / rank
                    break
            
            # Добавляем к итогам
            total_precision += precision
            total_recall += recall
            total_hit += hit
            total_mrr += mrr
            valid_queries += 1
            
            # Выводим детали для первых 3 вопросов
            if i < 3:
                print(f"\nВопрос {i+1}: '{query[:50]}...'")
                print(f"Правильные ID: {list(correct_ids)}")
                print(f"Найденные ID: {found_ids}")
                print(f"Совпадения: {correct_found}")
                print(f"Несовпадения: {[id for id in found_ids if id not in correct_ids]}")
                print(f"Precision@{k}: {precision:.3f}, Recall@{k}: {recall:.3f}")
        
        # Средние метрики
        if valid_queries > 0:
            avg_precision = total_precision / valid_queries
            avg_recall = total_recall / valid_queries
            avg_hit = total_hit / valid_queries
            avg_mrr = total_mrr / valid_queries
            
            # F1-score
            f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
            
            print(f"\n{'='*80}")
            print(f"ИТОГО ДЛЯ TOP-{k} (на {valid_queries} вопросах):")
            print(f"  Precision@{k}: {avg_precision:.3f}")
            print(f"  Recall@{k}:    {avg_recall:.3f}")
            print(f"  F1@{k}:        {f1:.3f}")
            print(f"  Hit Rate@{k}:  {avg_hit:.3f}")
            print(f"  MRR@{k}:       {avg_mrr:.3f}")
            print(f"{'='*80}")
            
            results.append({
                'k': k,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': f1,
                'hit_rate': avg_hit,
                'mrr': avg_mrr
            })
    
    return results


evaluation_results = evaluate_search(enhanced_time_dense_search, k_values=[1, 3, 5, 10])

print(f"\n{'='*80}")
print("СВОДНАЯ ТАБЛИЦА МЕТРИК")
print("="*80)
print(f"{'K':>4} {'Precision':>10} {'Recall':>10} {'F1':>10} {'Hit Rate':>10} {'MRR':>10}")
print("-" * 80)

for res in evaluation_results:
    print(f"{res['k']:>4} "
          f"{res['precision']:>10.3f} "
          f"{res['recall']:>10.3f} "
          f"{res['f1']:>10.3f} "
          f"{res['hit_rate']:>10.3f} "
          f"{res['mrr']:>10.3f}")

print("="*80)
safe_model_name = model_name.replace('/', '_') 
table_file = f"/kaggle/working/metrics_{safe_model_name}.txt"
with open(table_file, 'w', encoding='utf-8') as f:
    # Простая таблица
    f.write("Метрики поиска по dense эмбеддингам\n")
    f.write("="*60 + "\n")
    f.write("K  Precision  Recall     F1        HitRate   MRR\n")
    f.write("-" * 60 + "\n")
    
    for res in evaluation_results:
        f.write(f"{res['k']:2}  {res['precision']:8.3f}  {res['recall']:8.3f}  "
                f"{res['f1']:8.3f}  {res['hit_rate']:8.3f}  {res['mrr']:8.3f}\n")
    
    f.write("="*60 + "\n")

print(f"Таблица сохранена в {table_file}")

# 7. Детальный анализ для всех вопросов (сохранение в файл)
print("\nСОХРАНЕНИЕ ДЕТАЛЬНЫХ РЕЗУЛЬТАТОВ...")

detailed_results = []

for i, question_data in enumerate(questions):
    query = question_data.get("question", "")
    correct_ids = set(question_data.get("id", []))
    
    if not correct_ids:
        continue
    
    # Ищем для Top-5
    found_ids = enhanced_time_dense_search(query, k=5)
    
    # Определяем совпадения
    matches = [id for id in found_ids if id in correct_ids]
    non_matches = [id for id in found_ids if id not in correct_ids]
    
    # Precision для этого запроса
    precision_5 = len(matches) / 5 if found_ids else 0
    
    detailed_results.append({
        'query_id': i + 1,
        'query': query,
        'correct_ids': list(correct_ids),
        'found_ids': found_ids,
        'matches': matches,
        'non_matches': non_matches,
        'precision_5': precision_5,
        'num_matches': len(matches)
    })
    
    # Выводим детали для первых 5 вопросов
    if i < 5:
        print(f"\nВопрос {i+1}:")
        print(f"  Запрос: '{query[:60]}...'")
        print(f"  Правильные ID: {list(correct_ids)}")
        print(f"  Найденные ID: {found_ids}")
        print(f"  ✓ Совпали: {matches}")
        print(f"  ✗ Не совпали: {non_matches}")
        print(f"  Precision@5: {precision_5:.3f}")

safe_model_name = model_name.replace('/', '_') 
output_file = f"/kaggle/working/search_evaluation_details_{safe_model_name}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(detailed_results, f, indent=2, ensure_ascii=False)

print(f"\n✓ Детальные результаты сохранены в {output_file}")

# 8. Статистика совпадений
print(f"\n{'='*80}")
print("СТАТИСТИКА СОВПАДЕНИЙ (Top-5)")
print("="*80)

total_questions = len([q for q in questions if q.get('id')])
total_positions = total_questions * 5
total_matches = sum(res['num_matches'] for res in detailed_results)
total_non_matches = total_positions - total_matches

print(f"Всего вопросов с ответами: {total_questions}")
print(f"Всего проверок (вопросов × Top-5): {total_positions}")
print(f"Всего совпадений: {total_matches}")
print(f"Всего несовпадений: {total_non_matches}")
print(f"Процент совпадений: {total_matches/total_positions*100:.1f}%")

# Распределение по количеству совпадений
matches_dist = {}
for res in detailed_results:
    num_matches = res['num_matches']
    matches_dist[num_matches] = matches_dist.get(num_matches, 0) + 1

print(f"\nРаспределение совпадений на вопрос:")
for num in sorted(matches_dist.keys()):
    count = matches_dist[num]
    percentage = count / total_questions * 100
    print(f"  {num} совпадений: {count} вопросов ({percentage:.1f}%)")

print("="*80)

Статей: 5488
Вопросов: 60
Эмбеддинги: (5488, 1024)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Batches:   0%|          | 0/172 [00:00<?, ?it/s]

Эмбеддинги названий: (5488, 1024)

ОЦЕНКА ПОИСКА ПО DENSE ЭМБЕДДИНГАМ

ТОП-1 РЕЗУЛЬТАТЫ:
--------------------------------------------------------------------------------

Вопрос 1: 'What are the features that GPT-5 can provide compa...'
Правильные ID: ['13ed2979-8642-5bcd-809d-8897177f0c3d', '81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c', '9a0df61d-4783-51f2-acc4-e66f52cd0813', '69e1be8c-f789-5de3-8c1a-c89f92639178', '26a13d66-152b-5969-93e9-41e0b7b5c9aa']
Найденные ID: ['81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c']
Совпадения: ['81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c']
Несовпадения: []
Precision@1: 1.000, Recall@1: 0.200

Вопрос 2: 'What new technologies in processor manufacturing w...'
Правильные ID: ['8b678d66-56a3-55b5-9eb0-bcf11368c852', 'eb17f839-22c4-5c8e-ab21-32f423d32aa2', '7ab20568-f68c-5cfe-a657-14c93f76771c', 'ed6f3aa0-07bc-519e-b76a-aa79dc04b0a7', '821d70b8-720d-5430-8fe5-3f679ffdfdc7']
Найденные ID: ['ed6f3aa0-07bc-519e-b76a-aa79dc04b0a7']
Совпадения: ['ed6f3aa0-07bc-519e-b76a-aa79dc04b0

In [13]:
found_ids = enhanced_time_dense_search("What are AI agent systems, and how do they change the way automation is approached?", k=5)
print(found_ids)

Преобразование 83 текстов в sparse эмбеддинги...


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## BM25

In [7]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re
from typing import List, Dict
import numpy as np

# Скачиваем необходимые ресурсы NLTK
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
except:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('omw-eng')  # Open Multilingual WordNet

class TextPreprocessor:
    def __init__(self, language='english'):
        """
        Инициализация препроцессора текста
        
        Args:
            language: язык текста ('english', 'russian')
        """
        self.language = language
        
        # Инициализируем стеммер и лемматизатор
        if language == 'english':
            self.stemmer = SnowballStemmer('english')
            self.lemmatizer = WordNetLemmatizer()
            self.stop_words = set(stopwords.words('english'))
    
    def preprocess(self, text: str, use_stemming: bool = True, 
                   use_lemmatization: bool = True, remove_stopwords: bool = True) -> List[str]:
        """
        Препроцессинг текста
        
        Args:
            text: исходный текст
            use_stemming: применять стемминг
            use_lemmatization: применять лемматизацию
            remove_stopwords: удалять стоп-слова
            
        Returns:
            Список обработанных токенов
        """
        if not text:
            return []
        
        # 1. Приведение к нижнему регистру
        text = text.lower()
        
        # 2. Удаление HTML-тегов и специальных символов
        text = re.sub(r'<[^>]+>', ' ', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        
        # 3. Токенизация
        tokens = nltk.word_tokenize(text)
        
        # 4. Фильтрация токенов
        processed_tokens = []
        for token in tokens:
            # Пропускаем числа и очень короткие токены
            if token.isdigit() or len(token) < 2:
                continue
                
            # Удаление стоп-слов
            if remove_stopwords and token in self.stop_words:
                continue
            
            # Лемматизация (если доступна и включена)
            if use_lemmatization and self.lemmatizer:
                if self.language == 'english':
                    # Для английского определяем часть речи
                    token = self.lemmatizer.lemmatize(token, pos='v')  # глагол
                    token = self.lemmatizer.lemmatize(token, pos='n')  # существительное
                    token = self.lemmatizer.lemmatize(token, pos='a')  # прилагательное
                    token = self.lemmatizer.lemmatize(token, pos='r')  # наречие
            
            # Стемминг
            if use_stemming:
                token = self.stemmer.stem(token)
            
            processed_tokens.append(token)
        
        return processed_tokens
    
    def preprocess_batch(self, texts: List[str], **kwargs) -> List[List[str]]:
        """Препроцессинг батча текстов"""
        return [self.preprocess(text, **kwargs) for text in texts]


class BM25WithPreprocessing:
    def __init__(self, k1: float = 1.5, b: float = 0.75, language: str = 'english'):
        """
        BM25 с препроцессингом текста
        
        Args:
            k1: параметр регулирования частоты термина
            b: параметр регулирования длины документа
            language: язык текста
        """
        self.k1 = k1
        self.b = b
        self.preprocessor = TextPreprocessor(language)
        
        # Статистики корпуса
        self.doc_freq = {}  # DF(t)
        self.term_freq = []  # TF(t, d)
        self.doc_lengths = []  # |d|
        self.avg_doc_length = 0.0
        self.num_docs = 0
        self.vocab = set()
        
        # Настройки препроцессинга
        self.use_stemming = True
        self.use_lemmatization = True
        self.remove_stopwords = True
        
    def set_preprocessing_options(self, use_stemming: bool = True, 
                                  use_lemmatization: bool = True, 
                                  remove_stopwords: bool = True):
        """Настройка опций препроцессинга"""
        self.use_stemming = use_stemming
        self.use_lemmatization = use_lemmatization
        self.remove_stopwords = remove_stopwords
    
    def fit(self, documents: List[str]) -> 'BM25WithPreprocessing':
        """
        Обучение BM25 на корпусе документов с препроцессингом
        """
        print(f"Обучение BM25 с препроцессингом на {len(documents)} документах...")
        
        # Препроцессинг всех документов
        print("Препроцессинг документов...")
        processed_docs = self.preprocessor.preprocess_batch(
            documents,
            use_stemming=self.use_stemming,
            use_lemmatization=self.use_lemmatization,
            remove_stopwords=self.remove_stopwords
        )
        
        self.num_docs = len(processed_docs)
        self.doc_freq = {}
        self.term_freq = []
        self.doc_lengths = []
        
        total_length = 0
        
        for doc_tokens in processed_docs:
            # Длина документа
            doc_len = len(doc_tokens)
            self.doc_lengths.append(doc_len)
            total_length += doc_len
            
            # Частоты терминов в документе
            term_counts = {}
            for token in doc_tokens:
                term_counts[token] = term_counts.get(token, 0) + 1
            self.term_freq.append(term_counts)
            
            # Обновляем document frequency
            for term in term_counts.keys():
                self.doc_freq[term] = self.doc_freq.get(term, 0) + 1
                self.vocab.add(term)
        
        # Средняя длина документа
        self.avg_doc_length = total_length / self.num_docs if self.num_docs > 0 else 0
        
        print(f"Обучение завершено:")
        print(f"  - Документов: {self.num_docs}")
        print(f"  - Уникальных терминов (после препроцессинга): {len(self.vocab)}")
        print(f"  - Средняя длина документа: {self.avg_doc_length:.1f} токенов")
        
        return self
    
    def tokenize_query(self, query: str) -> List[str]:
        """Токенизация запроса с тем же препроцессингом"""
        return self.preprocessor.preprocess(
            query,
            use_stemming=self.use_stemming,
            use_lemmatization=self.use_lemmatization,
            remove_stopwords=self.remove_stopwords
        )
    
    def idf(self, term: str) -> float:
        """Inverse Document Frequency"""
        import math
        if term not in self.doc_freq:
            return 0.0
        
        df_t = self.doc_freq[term]
        numerator = self.num_docs - df_t + 0.5
        denominator = df_t + 0.5
        
        if denominator <= 0 or numerator <= 0:
            return 0.0
        
        return math.log(numerator / denominator + 1)
    
    def tf_component(self, tf: int, doc_len: int) -> float:
        """Term Frequency компонент"""
        length_norm = 1 - self.b + self.b * (doc_len / self.avg_doc_length)
        numerator = tf * (self.k1 + 1)
        denominator = tf + self.k1 * length_norm
        return numerator / denominator if denominator > 0 else 0.0
    
    def search(self, query: str, k: int = 10) -> tuple:
        """
        Поиск по запросу с препроцессингом
        """
        # Токенизация запроса
        query_tokens = self.tokenize_query(query)
        
        if not query_tokens:
            return [], []
        
        # Подсчет частот терминов в запросе
        query_term_counts = {}
        for token in query_tokens:
            query_term_counts[token] = query_term_counts.get(token, 0) + 1
        
        # Вычисление скоров
        scores = np.zeros(self.num_docs)
        
        for doc_idx in range(self.num_docs):
            doc_score = 0.0
            doc_len = self.doc_lengths[doc_idx]
            
            for term, query_tf in query_term_counts.items():
                idf_score = self.idf(term)
                
                if idf_score > 0:
                    doc_tf = self.term_freq[doc_idx].get(term, 0)
                    
                    if doc_tf > 0:
                        tf_component = self.tf_component(doc_tf, doc_len)
                        doc_score += idf_score * tf_component
            
            scores[doc_idx] = doc_score
        
        # Получаем топ-K
        if k > len(scores):
            k = len(scores)
        
        top_indices = np.argpartition(-scores, k)[:k]
        sorted_indices = top_indices[np.argsort(-scores[top_indices])]
        sorted_scores = scores[sorted_indices]
        
        return sorted_indices.tolist(), sorted_scores.tolist()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading omw-eng: Package 'omw-eng' not found in
[nltk_data]     index


In [8]:
def train_bm25_with_preprocessing(articles: List[Dict], 
                                 use_stemming: bool = True,
                                 use_lemmatization: bool = True,
                                 remove_stopwords: bool = True,
                                 language: str = 'english') -> tuple:
    """
    Обучение BM25 на статьях с препроцессингом
    
    Args:
        articles: список статей
        use_stemming: применять стемминг
        use_lemmatization: применять лемматизацию
        remove_stopwords: удалять стоп-слова
        language: язык текста
        
    Returns:
        Обученная модель BM25 и метаданные
    """
    print("="*80)
    print("ОБУЧЕНИЕ BM25 С ПРЕПРОЦЕССИНГОМ")
    print("="*80)
    
    article_texts = []
    article_metadata = []
    
    for article in articles:
        title = article.get('title', '')
        text = article.get('text', '')
        combined = f"{title}. {text}"
        article_texts.append(combined)

        article_metadata.append({
            'id': article.get('id', ''),
            'title': title,
            'published_time': article.get('published_time', ''),
            'original_text_preview': text[:200] if text else ''
        })
    
    print(f"Подготовлено {len(article_texts)} статей для обучения")
    print(f"Настройки препроцессинга:")
    print(f"  - Стемминг: {'ВКЛ' if use_stemming else 'ВЫКЛ'}")
    print(f"  - Лемматизация: {'ВКЛ' if use_lemmatization else 'ВЫКЛ'}")
    print(f"  - Удаление стоп-слов: {'ВКЛ' if remove_stopwords else 'ВЫКЛ'}")
    print(f"  - Язык: {language}")
    
    # Создаем и обучаем модель
    bm25_model = BM25WithPreprocessing(k1=1.5, b=0.75, language=language)
    bm25_model.set_preprocessing_options(
        use_stemming=use_stemming,
        use_lemmatization=use_lemmatization,
        remove_stopwords=remove_stopwords
    )
    
    bm25_model.fit(article_texts)
    
    return bm25_model, article_metadata


class BM25SearchSystem:
    def __init__(self, bm25_model, article_metadata, article_id_to_idx):
        """
        Система поиска на основе BM25
        
        Args:
            bm25_model: обученная модель BM25
            article_metadata: метаданные статей
            article_id_to_idx: маппинг ID -> индекс
        """
        self.bm25 = bm25_model
        self.article_metadata = article_metadata
        self.id_to_idx = article_id_to_idx
        self.idx_to_id = {idx: aid for aid, idx in article_id_to_idx.items()}
        
    def search(self, query: str, k: int = 10) -> List[Dict]:
        """
        Поиск статей по запросу
        
        Args:
            query: поисковый запрос
            k: количество результатов
            
        Returns:
            Список найденных статей с метаданными
        """
        indices, scores = self.bm25.search(query, k)

        results = []
        for idx, score in zip(indices, scores):
            if idx < len(self.article_metadata):
                metadata = self.article_metadata[idx]
                result = {
                    'id': metadata['id'],
                    'title': metadata['title'],
                    'score': float(score),
                    'published_time': metadata.get('published_time', ''),
                    'rank': len(results) + 1
                }
                results.append(result)
        
        return results
    
    def search_ids(self, query: str, k: int = 10) -> List[str]:
        """
        Поиск только ID статей
        
        Args:
            query: поисковый запрос
            k: количество результатов
            
        Returns:
            Список ID статей
        """
        indices, _ = self.bm25.search(query, k)
        article_ids = []
        
        for idx in indices:
            if idx in self.idx_to_id:
                article_ids.append(self.idx_to_id[idx])
            elif 0 <= idx < len(self.article_metadata):
                article_ids.append(self.article_metadata[idx]['id'])
        
        return article_ids

<function __main__.train_bm25_with_preprocessing(articles: List[Dict], use_stemming: bool = True, use_lemmatization: bool = True, remove_stopwords: bool = True, language: str = 'english') -> tuple>

In [9]:
with open('/kaggle/input/crunch/techcrunch_ai_5488_articles_20260112_1535.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)

print(f"Загружено {len(articles)} статей")

bm25_model, article_metadata = train_bm25_with_preprocessing(articles)

article_id_to_idx = {}
for idx, article in enumerate(articles):
    article_id = article.get('id', str(idx))
    article_id_to_idx[article_id] = idx

bm25_search = BM25SearchSystem(bm25_model, article_metadata, article_id_to_idx)

Загружено 5488 статей
ОБУЧЕНИЕ BM25 С ПРЕПРОЦЕССИНГОМ
Подготовлено 5488 статей для обучения
Настройки препроцессинга:
  - Стемминг: ВКЛ
  - Лемматизация: ВКЛ
  - Удаление стоп-слов: ВКЛ
  - Язык: english
Обучение BM25 с препроцессингом на 5488 документах...
Препроцессинг документов...
Обучение завершено:
  - Документов: 5488
  - Уникальных терминов (после препроцессинга): 60846
  - Средняя длина документа: 429.5 токенов


In [10]:
def save_embeddings_minimal(bm25_model):

    from scipy import sparse
    
    num_docs = bm25_model.num_docs
    vocab = list(bm25_model.doc_freq.keys())
    term_idx = {t:i for i,t in enumerate(vocab)}
    
    rows, cols, data = [], [], []
    
    for d in range(num_docs):
        for t, tf in bm25_model.term_freq[d].items():
            if t in term_idx:
                ti = term_idx[t]
                tfidf = bm25_model.tf_component(tf, bm25_model.doc_lengths[d]) * bm25_model.idf(t)
                rows.append(d)
                cols.append(ti)
                data.append(tfidf)
    
    matrix = sparse.csr_matrix((data, (rows, cols)), shape=(num_docs, len(vocab)))
    print(matrix.shape[1])
    sparse.save_npz('/kaggle/working/bm25_matrix.npz', matrix)
    
    return '/kaggle/working/bm25_matrix.npz'

matrix_file = save_embeddings_minimal(bm25_model)

60846


In [11]:
def bm25_search_function(query: str, k: int = 10) -> List[str]:
    """Функция поиска BM25 для интеграции с вашей системой оценки"""
    return bm25_search.search_ids(query, k=k)


def evaluate_bm25_search(search_func, k_values=[1, 3, 5, 10]):
    """Оценка BM25 поиска"""
    print(f"\n{'='*60}")
    print("ОЦЕНКА BM25 ПОИСКА")
    print(f"{'='*60}")
    
    with open('/kaggle/input/crunch2/questions.json', 'r', encoding='utf-8') as f:
        questions = json.load(f)
    
    results = []
    
    for k in k_values:
        print(f"\nТОП-{k} РЕЗУЛЬТАТЫ:")
        print("-" * 80)
        
        total_precision = 0
        total_recall = 0
        total_hit = 0
        total_mrr = 0
        valid_queries = 0
        
        for i, question_data in enumerate(questions):
            query = question_data.get("question", "")
            correct_ids = set(question_data.get("id", []))
            
            if not correct_ids:
                continue

            found_ids = search_func(query, k=k)

            correct_found = [id for id in found_ids if id in correct_ids]
            
            precision = len(correct_found) / k if k > 0 else 0
            recall = len(correct_found) / len(correct_ids) if correct_ids else 0
            hit = 1 if len(correct_found) > 0 else 0
            
            mrr = 0
            for rank, found_id in enumerate(found_ids, 1):
                if found_id in correct_ids:
                    mrr = 1.0 / rank
                    break
            
            total_precision += precision
            total_recall += recall
            total_hit += hit
            total_mrr += mrr
            valid_queries += 1
            
            if i < 2:
                print(f"\nВопрос {i+1}: '{query[:50]}...'")
                print(f"  Правильные: {list(correct_ids)}")
                print(f"  Найденные: {found_ids}")
                print(f"  Precision@{k}: {precision:.3f}, Recall@{k}: {recall:.3f}")
        
        if valid_queries > 0:
            avg_precision = total_precision / valid_queries
            avg_recall = total_recall / valid_queries
            avg_hit = total_hit / valid_queries
            avg_mrr = total_mrr / valid_queries
            
            f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
            
            print(f"\n{'='*80}")
            print(f"ИТОГО ДЛЯ TOP-{k} (на {valid_queries} вопросах):")
            print(f"  Precision@{k}: {avg_precision:.3f}")
            print(f"  Recall@{k}:    {avg_recall:.3f}")
            print(f"  F1@{k}:        {f1:.3f}")
            print(f"  Hit Rate@{k}:  {avg_hit:.3f}")
            print(f"  MRR@{k}:       {avg_mrr:.3f}")
            print(f"{'='*80}")
            
            results.append({
                'k': k,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': f1,
                'hit_rate': avg_hit,
                'mrr': avg_mrr
            })
    
    return results


print("\n" + "="*80)
print("НАЧАЛО ОЦЕНКИ BM25")
print("="*80)

evaluation_results = evaluate_bm25_search(bm25_search_function, k_values=[1, 3, 5, 10])

print(f"\n{'='*80}")
print("СВОДНАЯ ТАБЛИЦА МЕТРИК BM25")
print("="*80)
print(f"{'K':>4} {'Precision':>10} {'Recall':>10} {'F1':>10} {'Hit Rate':>10} {'MRR':>10}")
print("-" * 80)

for res in evaluation_results:
    print(f"{res['k']:>4} "
          f"{res['precision']:>10.3f} "
          f"{res['recall']:>10.3f} "
          f"{res['f1']:>10.3f} "
          f"{res['hit_rate']:>10.3f} "
          f"{res['mrr']:>10.3f}")

print("="*80)

with open('/kaggle/working/bm25_metrics.txt', 'w', encoding='utf-8') as f:
    f.write("Метрики BM25 поиска\n")
    f.write("="*60 + "\n")
    f.write("K  Precision  Recall     F1        HitRate   MRR\n")
    f.write("-" * 60 + "\n")
    
    for res in evaluation_results:
        f.write(f"{res['k']:2}  {res['precision']:8.3f}  {res['recall']:8.3f}  "
                f"{res['f1']:8.3f}  {res['hit_rate']:8.3f}  {res['mrr']:8.3f}\n")
    
    f.write("="*60 + "\n")

print("Результаты сохранены в /kaggle/working/bm25_metrics.txt")


НАЧАЛО ОЦЕНКИ BM25

ОЦЕНКА BM25 ПОИСКА

ТОП-1 РЕЗУЛЬТАТЫ:
--------------------------------------------------------------------------------

Вопрос 1: 'What are the features that GPT-5 can provide compa...'
  Правильные: ['81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c', '13ed2979-8642-5bcd-809d-8897177f0c3d', '9a0df61d-4783-51f2-acc4-e66f52cd0813', '26a13d66-152b-5969-93e9-41e0b7b5c9aa', '69e1be8c-f789-5de3-8c1a-c89f92639178']
  Найденные: ['424d6560-9035-5bba-aa27-771e679fce30']
  Precision@1: 0.000, Recall@1: 0.000

Вопрос 2: 'What new technologies in processor manufacturing w...'
  Правильные: ['821d70b8-720d-5430-8fe5-3f679ffdfdc7', '7ab20568-f68c-5cfe-a657-14c93f76771c', 'ed6f3aa0-07bc-519e-b76a-aa79dc04b0a7', 'eb17f839-22c4-5c8e-ab21-32f423d32aa2', '8b678d66-56a3-55b5-9eb0-bcf11368c852']
  Найденные: ['ed6f3aa0-07bc-519e-b76a-aa79dc04b0a7']
  Precision@1: 1.000, Recall@1: 0.200

ИТОГО ДЛЯ TOP-1 (на 60 вопросах):
  Precision@1: 0.200
  Recall@1:    0.038
  F1@1:        0.064
  Hit Rate@1: 